In [246]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [247]:
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [248]:
df=pd.read_csv("/content/fake_news_dataset.csv")

In [249]:
df.head()

,title,text,date,source,author,category,label
0,Foreign Democrat final.,more tax development both store agreement lawy...,2023-03-10,NY Times,Paula George,Politics,real
1,To offer down resource great point.,probably guess western behind likely next inve...,2022-05-25,Fox News,Joseph Hill,Politics,fake
2,Himself church myself carry.,them identify forward present success risk sev...,2022-09-01,CNN,Julia Robinson,Business,fake
3,You unit its should.,phone which item yard Republican safe where po...,2023-02-07,Reuters,Mr. David Foster DDS,Science,fake
4,Billion believe employee summer how.,wonder myself fact difficult course forget exa...,2023-04-03,CNN,Austin Walker,Technology,fake


In [250]:
df['date']=df.drop(columns='date',axis=1, inplace=True)

In [251]:
df.drop(columns='source', axis=1 , inplace=True)


In [252]:
df.head()

,title,text,author,category,label,date
0,Foreign Democrat final.,more tax development both store agreement lawy...,Paula George,Politics,real,None
1,To offer down resource great point.,probably guess western behind likely next inve...,Joseph Hill,Politics,fake,None
2,Himself church myself carry.,them identify forward present success risk sev...,Julia Robinson,Business,fake,None
3,You unit its should.,phone which item yard Republican safe where po...,Mr. David Foster DDS,Science,fake,None
4,Billion believe employee summer how.,wonder myself fact difficult course forget exa...,Austin Walker,Technology,fake,None


In [253]:
df.drop(columns='date',axis=1, inplace=True)

In [254]:
df.head()

,title,text,author,category,label
0,Foreign Democrat final.,more tax development both store agreement lawy...,Paula George,Politics,real
1,To offer down resource great point.,probably guess western behind likely next inve...,Joseph Hill,Politics,fake
2,Himself church myself carry.,them identify forward present success risk sev...,Julia Robinson,Business,fake
3,You unit its should.,phone which item yard Republican safe where po...,Mr. David Foster DDS,Science,fake
4,Billion believe employee summer how.,wonder myself fact difficult course forget exa...,Austin Walker,Technology,fake


In [255]:
df['label'].replace({'real': 1, 'fake': 0}, inplace=True)


/tmp/ipython-input-4180984749.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['label'].replace({'real': 1, 'fake': 0}, inplace=True)
/tmp/ipython-input-4180984749.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'].replace({'real': 1, 'fake': 0}, inplace=True)


In [256]:
df.shape

(20000, 5)

In [257]:
df.isnull().sum()

,0
title,0
text,0
author,1000
category,0
label,0


In [258]:
df=df.fillna('')

In [259]:
df.isnull().sum()

,0
title,0
text,0
author,0
category,0
label,0


In [260]:
df['content']=df['author']+' ' +df['title']

In [261]:
print(df.content)

0                     Paula George Foreign Democrat final.
1          Joseph Hill To offer down resource great point.
2              Julia Robinson Himself church myself carry.
3                Mr. David Foster DDS You unit its should.
4        Austin Walker Billion believe employee summer ...
                               ...                        
19995                         Gary Miles House party born.
19996    Maria Mcbride Though nation people maybe price...
19997     Kristen Franklin Yet exist with experience unit.
19998                  David Wise School wide itself item.
19999        James Peterson Offer chair cover senior born.
Name: content, Length: 20000, dtype: object


In [262]:
X=df.drop(['label'], axis=1)
y=df['label']

In [263]:
print(X)
print(y)

                                       title  \
0                    Foreign Democrat final.   
1        To offer down resource great point.   
2               Himself church myself carry.   
3                       You unit its should.   
4       Billion believe employee summer how.   
...                                      ...   
19995                      House party born.   
19996  Though nation people maybe price box.   
19997        Yet exist with experience unit.   
19998               School wide itself item.   
19999         Offer chair cover senior born.   

                                                    text  \
0      more tax development both store agreement lawy...   
1      probably guess western behind likely next inve...   
2      them identify forward present success risk sev...   
3      phone which item yard Republican safe where po...   
4      wonder myself fact difficult course forget exa...   
...                                                  ...   
199

In [264]:
port_stem=PorterStemmer()

In [265]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]', ' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [266]:
df['content']=df['content'].apply(stemming)

In [267]:
print(df['content'])

0                      paula georg foreign democrat final
1                   joseph hill offer resourc great point
2                             julia robinson church carri
3                                 mr david foster dd unit
4             austin walker billion believ employe summer
                               ...                       
19995                           gari mile hous parti born
19996    maria mcbride though nation peopl mayb price box
19997              kristen franklin yet exist experi unit
19998                         david wise school wide item
19999         jame peterson offer chair cover senior born
Name: content, Length: 20000, dtype: object


In [268]:
X=df['content'].values
y=df['label'].values

In [269]:
print(X)
print(y)

['paula georg foreign democrat final'
 'joseph hill offer resourc great point' 'julia robinson church carri' ...
 'kristen franklin yet exist experi unit' 'david wise school wide item'
 'jame peterson offer chair cover senior born']
[1 0 0 ... 1 0 0]


In [270]:
vectorizer=TfidfVectorizer(max_features=10000, ngram_range=(1,3),stop_words='english')
vectorizer.fit(df['content'])

X=vectorizer.transform(df['content'])

In [271]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 142782 stored elements and shape (20000, 10000)>
  Coords	Values
  (0, 3229)	0.3230188984775633
  (0, 3233)	0.571944191905508
  (0, 3917)	0.31633034321740966
  (0, 4030)	0.35299855456103546
  (0, 4179)	0.38575629231270886
  (0, 7118)	0.4416535406297843
  (1, 4251)	0.34047540791294395
  (1, 4535)	0.3634236210268278
  (1, 5238)	0.3291477077105655
  (1, 5241)	0.549500072846975
  (1, 6878)	0.3391462562106277
  (1, 7276)	0.3395858135219534
  (1, 7814)	0.33372521164763813
  (2, 2394)	0.36865236657763756
  (2, 2652)	0.3738760372037231
  (2, 5297)	0.463176331359564
  (2, 5301)	0.6026092952351498
  (2, 8019)	0.3829384608341644
  (3, 3085)	0.2806133444664392
  (3, 3096)	0.5168731180696984
  (3, 3154)	0.3378791128160659
  (3, 4055)	0.3842313958445138
  (3, 6602)	0.2607369501376606
  (3, 6618)	0.46193396110860874
  (3, 9475)	0.3330698555884235
  :	:
  (19996, 6004)	0.4066678131569186
  (19996, 6132)	0.3607802765897681
  (19996, 6138)	0.

In [272]:
X_train, X_test, y_train , y_test= train_test_split(X, y , test_size=0.1, stratify=y, random_state=2)

In [273]:
print(X.shape, X_train.shape,X_test.shape, y_train.shape, y_test.shape)

(20000, 10000) (18000, 10000) (2000, 10000) (18000,) (2000,)


In [281]:
for c in [0.01, 0.1, 1, 10]:
    model = LogisticRegression(C=c, penalty='l2', max_iter=500, class_weight='balanced')

In [282]:
model.fit(X_train, y_train)

LogisticRegression(C=10, class_weight='balanced', max_iter=500)

In [283]:
X_train_prediction = model.predict(X_train)
traning_data_accuracy=accuracy_score(X_train_prediction, y_train)

In [284]:
print("The accuracy on traning data is :", traning_data_accuracy)

The accuracy on traning data is : 0.7883333333333333


In [285]:
X_test_prediction = model.predict(X_test)
testing_data_accuracy=accuracy_score(X_test_prediction, y_test)

In [286]:
print("The accuracy on testing data is :", testing_data_accuracy)

The accuracy on testing data is : 0.515


In [287]:
print(df['label'].value_counts())


label
0    10056
1     9944
Name: count, dtype: int64
